In [1]:
# General Imports

import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import os,sys,datetime,time,math, warnings,itertools

import pandas as pd
from contextlib import contextmanager

# Snowpark imports
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window

c:\Users\z003mxpm\Anaconda3\lib\site-packages\snowflake\connector\options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [3]:
# Connection Parameters
connection_parameters = {
    'account':'shsitdl.west-europe.azure',
    'user':'jan-lucas.deinhard@siemens-healthineers.com',
    'authenticator':'externalbrowser',
    'role':'FR_CRMCLOUD_DEV',
    'database':'ACCESSLAYER',
    'schema':'AC_CRMCLOUD_APP_PRICINGANALYTICS_P',
    'warehouse':'W_CRMCLOUD_SMALL_P'
}

# Establish Connection
session = Session.builder.configs(connection_parameters).create()

cC = session.table("\"In Vivo Quote Pricing Overview Systems (PCI)\"")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [4]:
cC = cC.with_column('ct',F.lit(1))

In [5]:
cC_sampled = cC.sample(n=100000)

In [6]:
rf = pd.DataFrame(columns=['Field','Maximum Row Count'])

for ccol in cC_sampled.columns:
    print('Working on column {0}...'.format(ccol))
    df = pd.DataFrame(cC_sampled.group_by(F.col(ccol)).agg(F.sum('ct').alias('\"Total Count\"')).order_by('\"Total Count\"',ascending=False).collect()).head(1)
    rf = pd.concat([rf,pd.DataFrame([[ccol,df.iloc[0]['Total Count']]],columns=['Field','Maximum Row Count'])])

Working on column ACCOUNT_ID...
Working on column "ACCOUNT_ID (End User)"...
Working on column "ACCOUNT_ID (MainAccount)"...
Working on column "Account Country"...
Working on column "Account Country Name"...
Working on column "Account Currency"...
Working on column "Account Name"...
Working on column "Account Name (End User)"...
Working on column "Account Name (MainAccount)"...
Working on column "Base Price (AC) (System)"...
Working on column "Base Price (DC) (System)"...
Working on column "Base Price (GC) (FX)"...
Working on column "Base Price (GC) (FX) (System)"...
Working on column "Base Price (GC) (System)"...
Working on column "Base Price (LC) (System)"...
Working on column "Base Price (MCMN) (System)"...
Working on column "Business Type"...
Working on column "CDV (Flag)"...
Working on column "CKAP (Flag)"...
Working on column "DQ Pricing In Vivo (Comment)"...
Working on column "DQ Pricing In Vivo (Flag)"...
Working on column "DQ Unit Difference in Source Systems (Comment)"...
Wor

In [8]:
rf.sort_values(by='Maximum Row Count',ascending=True)

,Field,Maximum Row Count
0,"""OLI Number""",3
0,OLI_ID,3
0,"""Quote Number""",17
0,"""Oppty Name""",131
0,"""ACCOUNT_ID (End User)""",239
...,...,...
0,"""Large Project (Flag)""",99557
0,"""In Vivo Sales Area 2""",100000
0,"""Quote Active (Flag)""",100000
0,"""Relevant (QLI) (Flag)""",100000


In [9]:
session.close()